# Подготовка датасета для обучения модели

### Выполнил: Земнухов Вадим. Группа: DS-10

### Предобработка распарсенных данных

Проверим данные, которые нам удалось спарсить при помощи scrapy с сайта http://mosopen.ru/streets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('addresses.csv')

In [ ]:
df.head()

,address
0,"Улица Ремизова, дом 1"
1,"Улица Ремизова, дом 2"
2,"Улица Ремизова, дом 3, корпус 1"
3,"Улица Ремизова, дом 3, корпус 2"
4,"Улица Ремизова, дом 4"


In [ ]:
df.iloc[1450][0]

'Электродная улица, дом 8, строение 3'

In [ ]:
df.shape

(103048, 1)

In [ ]:
df.describe()

,address
count,103048
unique,102802
top,"8-й Микрорайон, корпус 834"
freq,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103048 entries, 0 to 103047
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  103048 non-null  object
dtypes: object(1)
memory usage: 805.2+ KB


In [ ]:
df[df.isnull() == True].count()

address    0
dtype: int64

Пропусков в данных нет. Проверим уникальные адреса.

In [ ]:
len(df['address'].unique())

102802

Можно заметить, что при парсинге некоторые адреса продублировались, но тем не менее большая часть из них является уникальной. Отбросим дубликаты в нашем датасете.

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

(102802, 1)

Теперь дубликаты отсутствуют.

### Подготовка данных для дальнейшего обучения модели

Планируется, что наша будущая модель будет в качестве исходных данных брать текст после распознания речи и преобразовывать его в формат для чтения, например для подачи в приложение навигатора или такси для построения маршрута. В этом случае все числительные должны быть преобразованы непосредственно в числа. Названия улиц при этом должны остаться без изменений.

В качестве целевой переменной будем использовать наши адреса без запятых и переведенных в нижний регистр. При необходимости приведения в формат для чтения (заглавные буквы в именах собственных, сокращения слов), можно будет осуществить пост обработку предсказанных результатов при помощи библиотеки re.

В качестве данных, по которым нужно будет сделать предсказание будем использовать нормализованный текст без знаков препинания, так как их наличие для навигатора не нужно при определении адреса.

In [ ]:
import re

df['address'] = df['address'].apply(lambda x: re.sub(",","",x))
df.head()

,address
0,Улица Ремизова дом 1
1,Улица Ремизова дом 2
2,Улица Ремизова дом 3 корпус 1
3,Улица Ремизова дом 3 корпус 2
4,Улица Ремизова дом 4


In [ ]:
df.tail()

,address
103043,Проспект Мира дом 27 строение 9
103044,Проспект Мира дом 27 строение 10
103045,Проспект Мира дом 27 строение 11
103046,Проспект Мира дом 28
103047,Проспект Мира дом 29


Запятые были удалены. Теперь приведем всё к нижнемы регистру.

In [ ]:
df['target'] = df['address'].apply(lambda x: x.lower())
df.head()

,address,target
0,Улица Ремизова дом 1,улица ремизова дом 1
1,Улица Ремизова дом 2,улица ремизова дом 2
2,Улица Ремизова дом 3 корпус 1,улица ремизова дом 3 корпус 1
3,Улица Ремизова дом 3 корпус 2,улица ремизова дом 3 корпус 2
4,Улица Ремизова дом 4,улица ремизова дом 4


Теперь проведем нормализацию текста для создания данных, по которым нужно будет обучаться и делать предсказание. Для этого воспользуемся библиотекой NeMo от Nvidia.

In [ ]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install --upgrade numba

BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=bd267d0a9f3280cf626e33868c6ddc0a641b341aa2fc89d0dda08d568372e472
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 sox
0 upgraded, 6 newly installed, 0 to remove and 15 not upgraded.
Need to get 513 kB of archives.
After this operation, 1,564 kB of additional disk space will be use

In [ ]:
from nemo_text_processing.text_normalization.normalize import Normalizer

In [ ]:
normalizer = Normalizer(input_case = 'cased' , lang='ru', deterministic=False)

Нормализатор в NeMo для русского языка может выдавать несколько вариантов нормализованного текста. Проверим, как это работает на адресах, чтобы принять решение о том, как обрабатывать наш датасет.

In [ ]:
def written_to_spoken(written):
  return normalizer.normalize(written, verbose=False)

In [ ]:
import numpy as np

In [ ]:
norm_test = np.array(list(df['target'][:100]))
res_test = {}
for address in norm_test:
  res_test[address] = written_to_spoken(address)

print(res_test)

{'улица ремизова дом 1': 'улица ремизова дом одно', 'улица ремизова дом 2': 'улица ремизова дом две', 'улица ремизова дом 3 корпус 1': 'улица ремизова дом три корпус одно', 'улица ремизова дом 3 корпус 2': 'улица ремизова дом три корпус две', 'улица ремизова дом 4': 'улица ремизова дом четыре', 'улица ремизова дом 4 строение 1': 'улица ремизова дом четыре строение одно', 'улица ремизова дом 5': 'улица ремизова дом пять', 'улица ремизова дом 6': 'улица ремизова дом шесть', 'улица ремизова дом 6 строение 1': 'улица ремизова дом шесть строение одно', 'улица ремизова дом 7': 'улица ремизова дом семь', 'улица ремизова дом 8': 'улица ремизова дом восемь', 'улица ремизова дом 8 а строение 10': 'улица ремизова дом восемь а строение десять', 'улица ремизова дом 9': 'улица ремизова дом девять', 'улица ремизова дом 10': 'улица ремизова дом десять', 'улица ремизова дом 11 корпус 1': 'улица ремизова дом одиннадцать корпус одно', 'улица ремизова дом 11 корпус 2': 'улица ремизова дом одиннадцать корп

**В ряде случаев наблюдается некорректная нормализация:**

1) "1" принимает значение "одно" вместо "один".

2) "2" принимает значение "две" вместо "два".

3) Порядковые числительные нормализуются некорректно. Например "6-й" принимает значение "шесть й" после нормализации.

4) Некорректно нормализуются номера домов, в обозначении которых есть дробь. Например "1/19" принимает значение "первый девятнадцаным", "2/21" - "два два один".


**Решим данные проблемы следующими способами:**

1) После нормализации исправим все слова "одно" на "один" при помощи re.

2) После нормализации исправим все слова "две" на "два" при помощи re.

3) Порядковые числительные в основном встречаются в названиях улиц и как правило таких уникальных значений не очень много. Попробуем найти все порядковые числительные до нормализации текста и заменить их на верные текстовые значения при помощи библиотеки re.

4) Заменим все слэши "/" на значение " дробь ".

In [ ]:
df.head()

,address,target
0,Улица Ремизова дом 1,улица ремизова дом 1
1,Улица Ремизова дом 2,улица ремизова дом 2
2,Улица Ремизова дом 3 корпус 1,улица ремизова дом 3 корпус 1
3,Улица Ремизова дом 3 корпус 2,улица ремизова дом 3 корпус 2
4,Улица Ремизова дом 4,улица ремизова дом 4


In [ ]:
df['Normalized'] = df['target'].apply(lambda x: re.sub("/"," дробь ", x))

In [ ]:
df[df['target'].str.contains(r'/')]

,address,target,Normalized
71,Улица Кржижановского дом 1/19,улица кржижановского дом 1/19,улица кржижановского дом 1 дробь 19
72,Улица Кржижановского дом 2/21,улица кржижановского дом 2/21,улица кржижановского дом 2 дробь 21
73,Улица Кржижановского дом 2/30 корпус 7 строение 5,улица кржижановского дом 2/30 корпус 7 строение 5,улица кржижановского дом 2 дробь 30 корпус 7 с...
102,Нахимовский проспект дом 3/5,нахимовский проспект дом 3/5,нахимовский проспект дом 3 дробь 5
165,Улица Кржижановского дом 19/28,улица кржижановского дом 19/28,улица кржижановского дом 19 дробь 28
...,...,...,...
102389,Улица Большая Полянка дом 33/41 строение 1,улица большая полянка дом 33/41 строение 1,улица большая полянка дом 33 дробь 41 строение 1
102390,Улица Большая Полянка дом 33/41 строение 2,улица большая полянка дом 33/41 строение 2,улица большая полянка дом 33 дробь 41 строение 2
102391,Улица Большая Полянка дом 33/41 строение 3,улица большая полянка дом 33/41 строение 3,улица большая полянка дом 33 дробь 41 строение 3
102441,Югорский проезд дом 16/13,югорский проезд дом 16/13,югорский проезд дом 16 дробь 13


Теперь выясним какие порядковые числительные присутствуют в нашем датасете.

In [ ]:
df[df['target'].str.contains(r'-й')]

,address,target,Normalized
32,6-й Загородный проезд дом 1 строение 1,6-й загородный проезд дом 1 строение 1,6-й загородный проезд дом 1 строение 1
33,6-й Загородный проезд дом 5 строение 1,6-й загородный проезд дом 5 строение 1,6-й загородный проезд дом 5 строение 1
34,6-й Загородный проезд дом 5 строение 2,6-й загородный проезд дом 5 строение 2,6-й загородный проезд дом 5 строение 2
35,6-й Загородный проезд дом 5 строение 3,6-й загородный проезд дом 5 строение 3,6-й загородный проезд дом 5 строение 3
36,6-й Загородный проезд дом 5 строение 4,6-й загородный проезд дом 5 строение 4,6-й загородный проезд дом 5 строение 4
...,...,...,...
102142,1-й Голутвинский переулок дом 14 строение 2,1-й голутвинский переулок дом 14 строение 2,1-й голутвинский переулок дом 14 строение 2
102143,1-й Голутвинский переулок дом 14 строение 3,1-й голутвинский переулок дом 14 строение 3,1-й голутвинский переулок дом 14 строение 3
102144,1-й Голутвинский переулок дом 16 строение 1,1-й голутвинский переулок дом 16 строение 1,1-й голутвинский переулок дом 16 строение 1
102323,2-й Бабьегородский переулок дом 3,2-й бабьегородский переулок дом 3,2-й бабьегородский переулок дом 3


In [ ]:
df[df['target'].str.contains(r'-я')]

,address,target,Normalized
422,1-я Радиальная улица дом 1 строение 2,1-я радиальная улица дом 1 строение 2,1-я радиальная улица дом 1 строение 2
423,1-я Радиальная улица дом 1 строение 6,1-я радиальная улица дом 1 строение 6,1-я радиальная улица дом 1 строение 6
424,1-я Радиальная улица дом 1 строение 7,1-я радиальная улица дом 1 строение 7,1-я радиальная улица дом 1 строение 7
425,1-я Радиальная улица дом 1 А,1-я радиальная улица дом 1 а,1-я радиальная улица дом 1 а
426,1-я Радиальная улица дом 2 строение 2,1-я радиальная улица дом 2 строение 2,1-я радиальная улица дом 2 строение 2
...,...,...,...
100043,1-я Аэропортовская улица дом 5 строение 2,1-я аэропортовская улица дом 5 строение 2,1-я аэропортовская улица дом 5 строение 2
100044,1-я Аэропортовская улица дом 5 А,1-я аэропортовская улица дом 5 а,1-я аэропортовская улица дом 5 а
100045,1-я Аэропортовская улица дом 6,1-я аэропортовская улица дом 6,1-я аэропортовская улица дом 6
100046,1-я Аэропортовская улица дом 6 строение 5,1-я аэропортовская улица дом 6 строение 5,1-я аэропортовская улица дом 6 строение 5


In [ ]:
df[df['target'].str.contains(r'-е')]

,address,target,Normalized
72909,Покровский бульвар дом 11 строение 2-Е,покровский бульвар дом 11 строение 2-е,покровский бульвар дом 11 строение 2-е


Можно заметить, что такие значений довольно много. Поиск по содержанию подстроки '-е' показал, что в стлобце target также могут быть некоторые ошибки в обозначении. Их мы поправим чуть позже. Для начала выясним какие уникальные порядковые числительные у нас присутствуют.

In [ ]:
to_check = np.array(list(df[df['target'].str.contains(r'-й')]['Normalized']))

numbers = []

for i in to_check:
  check_1 = i.split()
  for j in check_1:
    if ('-й' in j) and (j not in numbers):
      numbers.append(j)

print(numbers)

['6-й', '2-й', '1-й', '3-й', '4-й', '5-й', '9-й', '8-й', '7-й', '18-й', '16-й', '15-й', '14-й', '12-й', '11-й', '10-й', '17-й']


Теперь также найдем порядковые числительные на '-я' и добавим их к нашему списку.

In [ ]:
to_check = np.array(list(df[df['target'].str.contains(r'-я')]['Normalized']))

for i in to_check:
  check_1 = i.split()
  for j in check_1:
    if ('-я' in j) and (j not in numbers):
      numbers.append(j)

print(numbers)

['6-й', '2-й', '1-й', '3-й', '4-й', '5-й', '9-й', '8-й', '7-й', '18-й', '16-й', '15-й', '14-й', '12-й', '11-й', '10-й', '17-й', '1-я', '2-я', '3-я', '4-я', '5-я', '9-я', '10-я', '8-я', '7-я', '6-я', '11-я', '12-я', '14-я', '13-я', '16-я', '15-я', 'тверской-ямской', 'тверская-ямская']


Очевидно, что последние 2 элемента списка лишние. Уберем их.

In [ ]:
numbers = numbers[:(len(numbers)-2)]

In [ ]:
print(*sorted(numbers))

1-й 1-я 10-й 10-я 11-й 11-я 12-й 12-я 13-я 14-й 14-я 15-й 15-я 16-й 16-я 17-й 18-й 2-й 2-я 3-й 3-я 4-й 4-я 5-й 5-я 6-й 6-я 7-й 7-я 8-й 8-я 9-й 9-я


In [ ]:
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("1-й","первый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("1-я","первая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("2-й","второй", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("2-я","вторая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("3-й","третий", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("3-я","третья", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("4-й","четвертый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("4-я","четвертая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("5-й","пятый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("5-я","пятая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("6-й","шестой", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("6-я","шестая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("7-й","седьмой", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("7-я","седьмая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("8-й","восьмой", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("8-я","восьмая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("9-й","девятый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("9-я","девятая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("10-й","десятый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("10-я","десятая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("11-й","одиннадцатый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("11-я","одиннадцатая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("12-й","двенадцатый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("12-я","двенадцатая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("13-я","тринадцатая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("14-й","четырнадцатый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("14-я","четырнадцатая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("15-й","пятнадцатый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("15-я","пятнадцатая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("16-й","шестнадцатый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("16-я","шестнадцатая", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("17-й","семнадцатый", x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub("18-й","восемнадцатый", x))

In [ ]:
df[df['Normalized'].str.contains(r'-й')]

,address,target,Normalized


In [ ]:
df[df['Normalized'].str.contains(r'-я')]

,address,target,Normalized
69650,1-й Тверской-Ямской переулок дом 11,1-й тверской-ямской переулок дом 11,первый тверской-ямской переулок дом 11
69651,1-й Тверской-Ямской переулок дом 14,1-й тверской-ямской переулок дом 14,первый тверской-ямской переулок дом 14
69652,1-й Тверской-Ямской переулок дом 14 строение 2,1-й тверской-ямской переулок дом 14 строение 2,первый тверской-ямской переулок дом 14 строение 2
69653,1-й Тверской-Ямской переулок дом 14 строение 3,1-й тверской-ямской переулок дом 14 строение 3,первый тверской-ямской переулок дом 14 строение 3
69654,1-й Тверской-Ямской переулок дом 14 строение 4,1-й тверской-ямской переулок дом 14 строение 4,первый тверской-ямской переулок дом 14 строение 4
...,...,...,...
69889,1-я Тверская-Ямская улица дом 30,1-я тверская-ямская улица дом 30,первая тверская-ямская улица дом 30
69890,1-я Тверская-Ямская улица дом 32,1-я тверская-ямская улица дом 32,первая тверская-ямская улица дом 32
69891,1-я Тверская-Ямская улица дом 34,1-я тверская-ямская улица дом 34,первая тверская-ямская улица дом 34
69892,1-я Тверская-Ямская улица дом 36 строение 1,1-я тверская-ямская улица дом 36 строение 1,первая тверская-ямская улица дом 36 строение 1


Можно заметить, что порядковых числительных в колонке Normalized не осталось. Однако при этом присутствуют дефисы между словами. После нормализации при помощи NeMo нужно будет проверить не осталось ли их в нашем тексте.

Теперь исправим ошибки в колонке target и как следствие Normalized.

In [ ]:
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('1-а', '1 а', x))
df['target'] = df['target'].apply(lambda x: re.sub('1-а', '1 а', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('1-б', '1 б', x))
df['target'] = df['target'].apply(lambda x: re.sub('1-б', '1 б', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('3-б', '3 б', x))
df['target'] = df['target'].apply(lambda x: re.sub('3-б', '3 б', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('1-в', '1 в', x))
df['target'] = df['target'].apply(lambda x: re.sub('1-в', '1 в', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('2-г', '2 г', x))
df['target'] = df['target'].apply(lambda x: re.sub('2-г', '2 г', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('3-г', '3 г', x))
df['target'] = df['target'].apply(lambda x: re.sub('3-г', '3 г', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('1-д', '1 д', x))
df['target'] = df['target'].apply(lambda x: re.sub('1-д', '1 д', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('2-е', '2 е', x))
df['target'] = df['target'].apply(lambda x: re.sub('2-е', '2 е', x))

In [ ]:
df[df['target'].str.contains(r'а-б')]

,address,target,Normalized
80044,Улица Воздвиженка дом 3/5 корпус А-Б строение 15,улица воздвиженка дом 3/5 корпус а-б строение 15,улица воздвиженка дом 3 дробь 5 корпус а-б стр...
80045,Улица Воздвиженка дом 3/5 корпус А-Б строение 18,улица воздвиженка дом 3/5 корпус а-б строение 18,улица воздвиженка дом 3 дробь 5 корпус а-б стр...


In [ ]:
df[df['target'].str.contains(r'б-в')]

,address,target,Normalized
71509,Селезнёвская улица дом 30 корпус Б-В,селезнёвская улица дом 30 корпус б-в,селезнёвская улица дом 30 корпус б-в


In [ ]:
df.shape

(102802, 3)

In [ ]:
df = df[df['target'] != 'улица воздвиженка дом 3/5 корпус а-б строение 15']
df = df[df['target'] != 'улица воздвиженка дом 3/5 корпус а-б строение 18']
df = df[df['target'] != 'селезнёвская улица дом 30 корпус б-в']

In [ ]:
df['Normalized'] = df['Normalized'].apply(written_to_spoken)

In [ ]:
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('одно', 'один', x))
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('две', 'два', x))

In [10]:
df[df['target'].str.contains(r'1строение')]['Normalized'].iloc[0]

'квартал самаркандский бульвар сто тридцать семь а корпус один строение четыре'

In [11]:
df[df['target'].str.contains(r'1строение')]['target'].iloc[0]

'квартал самаркандский бульвар 137а корпус 1строение 4'

In [14]:
df['target'] = df['target'].apply(lambda x: re.sub('строение', ' строение', x))
df['target'] = df['target'].apply(lambda x: re.sub('  ', ' ', x))

In [15]:
df[df['target'].str.contains(r'1строение')]['Normalized']

Series([], Name: Normalized, dtype: object)

In [21]:
df[df['Normalized'].str.contains(r'-')]

,address,target,Normalized
3094,Рубцовско-Дворцовая улица дом 1/3 строение 13,рубцовско-дворцовая улица дом 1/3 строение 13,рубцовско-дворцовая улица дом один дробь три с...
3095,Рубцовско-Дворцовая улица дом 1/3 строение 16,рубцовско-дворцовая улица дом 1/3 строение 16,рубцовско-дворцовая улица дом один дробь три с...
3096,Рубцовско-Дворцовая улица дом 1/3 строение 18,рубцовско-дворцовая улица дом 1/3 строение 18,рубцовско-дворцовая улица дом один дробь три с...
3097,Рубцовско-Дворцовая улица дом 1/3 строение 23,рубцовско-дворцовая улица дом 1/3 строение 23,рубцовско-дворцовая улица дом один дробь три с...
3098,Рубцовско-Дворцовая улица дом 1/3 строение 27,рубцовско-дворцовая улица дом 1/3 строение 27,рубцовско-дворцовая улица дом один дробь три с...
...,...,...,...
92936,Петровско-Разумовский проезд дом 24 корпус 3 с...,петровско-разумовский проезд дом 24 корпус 3 с...,петровско-разумовский проезд дом двадцать четы...
92937,Петровско-Разумовский проезд дом 24 корпус 4,петровско-разумовский проезд дом 24 корпус 4,петровско-разумовский проезд дом двадцать четы...
92938,Петровско-Разумовский проезд дом 24 корпус 5,петровско-разумовский проезд дом 24 корпус 5,петровско-разумовский проезд дом двадцать четы...
92939,Петровско-Разумовский проезд дом 24 корпус 15,петровско-разумовский проезд дом 24 корпус 15,петровско-разумовский проезд дом двадцать четы...


In [22]:
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('-', ' ', x))

In [23]:
df[df['Normalized'].str.contains(r'№')]

,address,target,Normalized
76368,Улица Льва Толстого дом 23/7№30,улица льва толстого дом 23/7№30,улица льва толстого дом двадцать три дробь 7№30
76369,Улица Льва Толстого дом 23/7№31,улица льва толстого дом 23/7№31,улица льва толстого дом двадцать три дробь 7№31
76370,Улица Льва Толстого дом 23/7№32,улица льва толстого дом 23/7№32,улица льва толстого дом двадцать три дробь 7№32


In [24]:
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('№', ' номер ', x))
df['target'] = df['target'].apply(lambda x: re.sub('№', ' № ', x))

In [27]:
df['target'] = df['target'].apply(lambda x: re.sub('/', ' / ', x))

In [28]:
df[df['target'].str.contains(r'/')]

,address,target,Normalized
71,Улица Кржижановского дом 1/19,улица кржижановского дом 1 / 19,улица кржижановского дом один дробь девятнадцать
72,Улица Кржижановского дом 2/21,улица кржижановского дом 2 / 21,улица кржижановского дом два дробь двадцать одна
73,Улица Кржижановского дом 2/30 корпус 7 строение 5,улица кржижановского дом 2 / 30 корпус 7 строе...,улица кржижановского дом два дробь тридцать ко...
102,Нахимовский проспект дом 3/5,нахимовский проспект дом 3 / 5,нахимовский проспект дом три дробь пять
165,Улица Кржижановского дом 19/28,улица кржижановского дом 19 / 28,улица кржижановского дом девятнадцать дробь дв...
...,...,...,...
102389,Улица Большая Полянка дом 33/41 строение 1,улица большая полянка дом 33 / 41 строение 1,улица большая полянка дом тридцать три дробь с...
102390,Улица Большая Полянка дом 33/41 строение 2,улица большая полянка дом 33 / 41 строение 2,улица большая полянка дом тридцать три дробь с...
102391,Улица Большая Полянка дом 33/41 строение 3,улица большая полянка дом 33 / 41 строение 3,улица большая полянка дом тридцать три дробь с...
102441,Югорский проезд дом 16/13,югорский проезд дом 16 / 13,югорский проезд дом шестнадцать дробь тринадцать


In [43]:
df[df['Normalized'].str.contains('одна')]

,address,target,Normalized
72,Улица Кржижановского дом 2/21,улица кржижановского дом 2 / 21,улица кржижановского дом два дробь двадцать одна
182,Улица Кржижановского дом 21/33 корпус 1,улица кржижановского дом 21 / 33 корпус 1,улица кржижановского дом двадцать одна дробь т...
183,Улица Кржижановского дом 21 А,улица кржижановского дом 21 а,улица кржижановского дом двадцать одна а
206,Нахимовский проспект дом 21,нахимовский проспект дом 21,нахимовский проспект дом двадцать одна
207,Нахимовский проспект дом 21 строение 1,нахимовский проспект дом 21 строение 1,нахимовский проспект дом двадцать одна строени...
...,...,...,...
102780,Палехская улица дом 131 А,палехская улица дом 131 а,палехская улица дом сто тридцать одна а
102820,Улица Красная Сосна дом 3 строение 21,улица красная сосна дом 3 строение 21,улица красная сосна дом три строение двадцать ...
102879,Улица Красная Сосна дом 31,улица красная сосна дом 31,улица красная сосна дом тридцать одна
103017,Проспект Мира дом 21,проспект мира дом 21,проспект мира дом двадцать одна


In [44]:
df['Normalized'] = df['Normalized'].apply(lambda x: re.sub('одна', 'один', x))

In [47]:
df[df['Normalized'].str.contains('одна')]

,address,target,Normalized


Теперь для перестраховки избавимся от возможных лишних пробелов в начале и конце текста.

In [16]:
df['target']=df['target'].apply(lambda x: x.strip())
df['Normalized']=df['Normalized'].apply(lambda x: x.strip())

In [55]:
import string

exclude = set(string.punctuation)

print(exclude)

{';', '=', '$', '?', '[', '{', '~', '&', '/', '^', ')', '-', '`', '(', ',', ']', "'", '.', '}', '\\', '#', '@', '>', '!', '"', '*', '+', '_', '|', '<', '%', ':'}


In [51]:
import string

exclude = set(string.punctuation)
df['Normalized'] = df['Normalized'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [59]:
exclude.discard('/')
exclude.discard('-')

In [61]:
df['target'] = df['target'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [62]:
df.head()

,address,target,Normalized
0,Улица Ремизова дом 1,улица ремизова дом 1,улица ремизова дом один
1,Улица Ремизова дом 2,улица ремизова дом 2,улица ремизова дом два
2,Улица Ремизова дом 3 корпус 1,улица ремизова дом 3 корпус 1,улица ремизова дом три корпус один
3,Улица Ремизова дом 3 корпус 2,улица ремизова дом 3 корпус 2,улица ремизова дом три корпус два
4,Улица Ремизова дом 4,улица ремизова дом 4,улица ремизова дом четыре


In [64]:
df.to_pickle('/content/drive/MyDrive/addresses_1.pkl')